# AI Agent - Castle Isolation

In this project, I engineered a series of sophisticated AI agents designed to compete in the game of Castle Isolation, demonstrating varying levels of strategic depth and computational efficiency. The core of these agents' capabilities was underpinned by a robust implementation of several well-established algorithms in game theory and artificial intelligence, including:

#### Minimax Algorithm 
This fundamental game-playing algorithm was employed to exhaustively explore possible moves and their implications, aiming to minimize the possible loss for a worst-case scenario.
#### Alpha-Beta Pruning
I enhanced the Minimax search efficiency by integrating Alpha-Beta pruning, which significantly reduces the number of nodes evaluated in the search tree, thereby optimizing performance without sacrificing decision quality.
#### Iterative Deepening
I applied this technique to combine the advantages of depth-first and breadth-first search strategies. It incrementally deepens the search allowing the algorithm to use more manageable iterations and improve time management by using results from previous iterations.
#### Evaluation Functions
I developed sophisticated evaluation functions that provide a heuristic assessment of game

---

### Disclaimer and Copyright Notice

1. The algorithms and techniques shown in this report were developed by me, Franz Adam. If code is used, please give credit accordingly. 
2. Since the implementation of algorithms overlaps with material in CS university courses, I advise current students to adhere to their academic institution's policies regarding integrity and plagiarism before continuing. 

## About the Game
The rules of Castle Isolation are a variation of the original Isolation game. In the original form of the game there are two players, each with their own game piece, and a 7-by-7 grid of squares. At the beginning of the game, the first player places their piece on any square. The second player follows suit, and places their piece on any one of the available squares. From that point on, the players alternate turns moving their piece like a queen in chess (any number of open squares vertically, horizontally, or diagonally). When the piece is moved, the square that was previously occupied is blocked, and cannot be used for the remainder of the game. The first player who is unable to move their queen loses.

In this variant called Castle Isolation, We have a 9-by-9 grid in this variation. There are 12 pre-defined squares which are blocked so as to create a castle-like structure. Neither players can move on or through squares that are blocked. The game ends when one of the players is not able to make a move and is trapped. For clarity, examine the scenario below:

### Rules visualized
Below is a depiction of the empty starting board position.

<div>
<img src="./img/starting_board.png" width="500"/>
</div>


Q1 places their queen on the board, and Q2 follows suit.

<div>
<img src="./img/castle_isolation_1.png" width="500"/>
</div>

Q1 moves vertically down for 3 blocks, consequently blocking some of Q2's potential future moves. The previous location of Q1 is also blocked as shown below.

<div>
<img src="./img/castle_isolation_2.png" width="500"/>
</div>

Q2 makes a one block move diagonally downward to the left simply blocking the previously held location.

<div>
<img src="./img/castle_isolation_3.png" width="500"/>
</div>

Both the players play a number of moves and the following is an example of when the game ends and Q2 wins. Q2 wins as there are no more possible moves for Q1!

<div>
<img src="./img/castle_isolation_4.png" width="500"/>
</div>


### For more clarity, You can try playing the game against the Random Player or yourself using the interactive tool below

In [1]:
# Following two lines make sure anything imported from .py scripts͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁 
# is automatically reloaded if edited & saved (e.g. local unit tests or players)͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
%load_ext autoreload
%autoreload 2
from board_viz import ReplayGame, InteractiveGame
from isolation import Board
from test_players import RandomPlayer

In [2]:
import time
from isolation import Board

## Evaluation Functions

The design of evaluation functions is pivotal in the development of game-playing AI agents. An effective evaluation function provides a heuristic estimate of the state's value, enabling the AI to make informed decisions by approximating the game's outcome from any given position.

For the game of Castle Isolation the evaluation function is crucial for assessing board configurations. Advanced functions often incorporate multiple heuristic indicators, such as the mobility of a piece (the number of available moves), control of the center, and potential pathways for trapping the opponent. These metrics reflect strategic nuances and foresight, allowing the AI to plan moves that maximize its positional advantage while minimizing future options for the opponent. 

One of the most intuitive and basic evaluation functions is the number of available legal moves for a given board configuration. This is what levels 1 and 2 use for my AI agent and what is depicted below. My level 3 agetn uses a specialized evaluation function which I will share with the reader when they successfully defeat my level 3 AI agent. 

The evaluation function is not just a tool for decision making in non-terminal states; it encapsulates the strategic essence of the game, translating abstract concepts like control, threat, and potential into actionable metrics that drive the AI’s performance towards victory.

```python 
class OpenMoveEvalFn:
    def score(self, game, my_player=None):
        """Score the current game state
        Basic Evaluation function that outputs a score equal to how many
        moves are open for AI player on the board minus how many moves
        are open for Opponent's player on the board.

            Args
                game (Board): The board and game state.
                my_player (Player object): This specifies which player you are.

            Returns:
                float: The current state's score. MyMoves-OppMoves.

            """
        player_moves = len(game.get_player_moves(my_player))
        opponent_moves = len(game.get_opponent_moves(my_player))

        ... # Here lies the hidden evaluation function. Defeat my level 3 AI agent and I will share it with you!
```

In [4]:
class CustomPlayer:
    """Player class that chooses a move using the evaluation function
    and applied algorithm."""

    def __init__(self, search_depth=4, eval_fn=OpenMoveEvalFn()):
        
        self.eval_fn = eval_fn
        self.search_depth = search_depth
    
    def move(self, game, time_left):
        """Called to determine one move by your agent
        Args:
            game (Board): The board and game state.
            time_left (function): Used to determine time left before timeout

        Returns:
            tuple: (int,int): Your best move
        """
        
        # Iterative deepening
        max_depth = 24
        best_moves = []
        
        for depth in range(1, max_depth + 1):     
            best_move, utility = alphabeta(self, game, time_left, depth = self.search_depth - self.search_depth + depth)
            best_moves.append(best_move)
    
            if best_moves[-1] == (-1,-1):
                return best_moves[-2]
                
        return best_move

    def utility(self, game, my_turn):
        """I handle special cases here (e.g. endgame)"""
        
        return self.eval_fn.score(game, self)

## Minimax
My implementation of the minimax algorithm.

```python
def minimax(player, game, time_left, depth, my_turn=True):
    """Implementation of the minimax algorithm.
    Args:
        player (CustomPlayer): This is the instantiation of CustomPlayer()
            that represents your agent. It is used to call anything you
            need from the CustomPlayer class (the utility() method, for example,
            or any class variables that belong to CustomPlayer()).
        game (Board): A board and game state.
        time_left (function): Used to determine time left before timeout
        depth: Used to track how deep you are in the search tree
        my_turn (bool): True if you are computing scores during your turn.

    Returns:
        (tuple, int): best_move, val
    """
    
    #Base Case:
    #if depth is reached or game is over (e.g., no more active moves for one player)
    if depth == 0:
        return (None, player.utility(game, my_turn))
    #return score of current position

    #Recursive Call
    if my_turn:
        best_cost = float("-inf")
        best_move = None
        
        for move in game.get_active_moves():
            _, eval = minimax(player, game.forecast_move(move)[0], time_left, depth - 1, my_turn = False)
            if eval > best_cost:
                best_cost = eval
                best_move = move
        return (best_move, best_cost)
        
    else: 
        best_cost = float("inf")
        best_move = None
        
        for move in game.get_active_moves():
            _, eval = minimax(player, game.forecast_move(move)[0], time_left, depth - 1, my_turn = True)
            if eval < best_cost:
                best_cost = eval
                best_move = move
        return (best_move, best_cost)
```

## Alphabeta Pruning
This is my implementation of the alphabeta pruning optimization technique for the Minimax algorithm.

``` python
def alphabeta(player, game, time_left, depth, alpha=float("-inf"), beta=float("inf"), my_turn=True):
    """Implementation of the alphabeta algorithm.
    
    Args:
        player (CustomPlayer): This is the instantiation of CustomPlayer() 
            that represents your agent. It is used to call anything you need 
            from the CustomPlayer class (the utility() method, for example, 
            or any class variables that belong to CustomPlayer())
        game (Board): A board and game state.
        time_left (function): Used to determine time left before timeout
        depth: Used to track how deep you are in the search tree
        alpha (float): Alpha value for pruning
        beta (float): Beta value for pruning
        my_turn (bool): True if you are computing scores during your turn.
        
    Returns:
        (tuple, int): best_move, val
    """
    
    if depth == 0 or len(game.get_active_moves()) == 0:
        return (None, player.utility(game, my_turn))

    if time_left() < 30: #Was 60 at 95pt submission 
        print(time_left())
        return (-1,-1), -100
    
    if my_turn:
        best_score, best_move = float("-inf"), None
        for move in game.get_active_moves():
            _, eval = alphabeta(player, game.forecast_move(move)[0], time_left, depth - 1, alpha, beta, my_turn = False)
            if eval == -100:
                return (-1,-1), -100
            if eval > best_score:
                best_score = eval
                best_move = move
            alpha = max(eval, alpha) 
            if beta <= alpha:
                break
        return (best_move, best_score)
        
    else: 
        best_score, best_move = float("inf"), None
        for move in game.get_active_moves():
            _, eval = alphabeta(player, game.forecast_move(move)[0], time_left, depth - 1, alpha, beta, my_turn = True)
            if eval < best_score:
                best_score = eval
                best_move = move
            beta = min(eval, beta)
            if beta <= alpha:
                break
        return (best_move, best_score)
        
```

## Iterative Deepening & Player Class
This class handles the application of the move finding algorithm as well as the iterative deepening optimization.

```python
class CustomPlayer:
    """Player class that chooses a move using the evaluation function
    and applied algorithm."""

    def __init__(self, search_depth=4, eval_fn=OpenMoveEvalFn()):
        
        self.eval_fn = eval_fn
        self.search_depth = search_depth
    
    def move(self, game, time_left):
        """Called to determine one move by your agent
        Args:
            game (Board): The board and game state.
            time_left (function): Used to determine time left before timeout

        Returns:
            tuple: (int,int): Your best move
        """
        
        # Iterative deepening step
        max_depth = 24
        best_moves = []
        
        for depth in range(1, max_depth + 1):     
            best_move, utility = alphabeta(self, game, time_left, depth = self.search_depth - self.search_depth + depth)
            best_moves.append(best_move)
    
            if best_moves[-1] == (-1,-1):
                return best_moves[-2]
                
        return best_move
```

## Challenge the AI agent
Ready to take on the battle against the machines? Challenge AI agent in a game of isolation here. Should you defeat level 3, I will share my special evaluation function with you!

In [14]:
ig = InteractiveGame(CustomPlayer(), show_legal_moves=True)

GridspecLayout(children=(Button(description=' ', layout=Layout(grid_area='widget001', height='auto', width='au…